In [31]:
import mdtraj as md
import numpy as np
np.set_printoptions(precision=4)
import pandas as pd
import tensorflow as tf
import math
from tqdm import tqdm
tfk = tf.keras
# # LEGACY CODE, now the gro files are filtered in the MD_sim pipeline so no need to select residues
# t = md.load('VAE/md_0_1.gro')
# rna_idx = t.top.select('resi < 30')
# rna = t.atom_slice(rna_idx)

In [1]:
! pwd

/work/donglab/ching.ki/ssRNA-MD/VAE/src


In [24]:
datafiles = [f"../data/filtered_sasdfb9_m{model}_{conc}_{ion}.gro" 
             for model in range(1,13)
             for conc in range(10, 60, 10)
             for ion in ["Na+", "MG"]]
rna = md.join([md.load(file) for file in datafiles])
n_residues = rna.top.n_residues
batch_residue_idx = [(i, i+3) for i in range(0,n_residues,3)]
atom_idx_list = []
for batch_start, batch_end in batch_residue_idx:
    atom_idx_list.append(rna.top.select(f'resi >= {batch_start} and resi < {batch_end}'))

In [27]:
def compute_and_pack_distance_in_array_frames(rna, pair_grid):
    # in distances_frames, the axes were (number of columns, number of frames, number of rows)
    # where number of columns and rows are both equal to the number of atoms
    distances_by_columns = np.array([md.compute_distances(rna, column) for column in pair_grid])
    # in axes_ordered_distances_frames, the axes are 
    # (number of frames, number of columns, number of rows)
    axes_ordered_distances_frames = distances_by_columns.transpose(1,0,2)
    packed_distances_frames = np.apply_along_axis(
        lambda arr: np.array_split(arr, len(arr)), 2, axes_ordered_distances_frames)
    return(packed_distances_frames)

# Given an 3D array (n_atom x n_atom x 1), n_atom is the number of atom in one batch of residue and 
# may vary depending on the number of atoms in each residue; return a 100x100x1 array either by 
# truncating or padding the given array
def trunc_or_pad_distance_array(distance_array, maxlen=100):
    if distance_array.shape[0] != distance_array.shape[1]:
        raise Exception("the first and second dimension of the given distance array does not match")
    n_atom = distance_array.shape[0]
    if n_atom < maxlen:
        pad_n_pre = int((maxlen-n_atom)/2)
        pad_n_post = (maxlen-n_atom)/2
        if pad_n_pre != pad_n_post:
            pad_n_post = math.ceil(pad_n_post)
        distance_array = np.pad(
            distance_array, 
            ((pad_n_pre, pad_n_post), 
             (pad_n_pre, pad_n_post),
             (0, 0)))
    elif n_atom > maxlen:
        trunc_n_pre = int((n_atom-maxlen)/2)
        trunc_n_post = (n_atom-maxlen)/2
        if trunc_n_pre != trunc_n_post:
            trunc_n_post = math.ceil(trunc_n_post)
        end_idx = int(n_atom - trunc_n_post)
        # truncate first dimension
        distance_array = distance_array[trunc_n_pre:end_idx]
        # truncate second dimension
        distance_array = np.array([col[trunc_n_pre:end_idx] for col in distance_array])    
    return distance_array

def preprocess_atom_set(rna, atom_idx):
    pair_meshgrid = np.meshgrid(atom_idx, np.transpose(atom_idx))
    pair_grid = np.dstack((pair_meshgrid[0], pair_meshgrid[1]))
    # TODO can be optimized to only calculate distance once for each pair but the computation quick so I am skipping for now
    pair_distance_array_frames = compute_and_pack_distance_in_array_frames(rna, pair_grid)
    formatted_pair_distance_array_frames = np.array(
        [trunc_or_pad_distance_array(frame, maxlen = 100) 
         for frame in pair_distance_array_frames])
    return formatted_pair_distance_array_frames

In [32]:
n_atom_idx_set = len(atom_idx_list)
n_atom_idx_set = 2
dataset_array = np.concatenate([preprocess_atom_set(rna, atom_idx_list[i]) for i in tqdm(range(n_atom_idx_set))])

100%|██████████| 2/2 [07:49<00:00, 234.88s/it]


In [33]:
n_atom_idx_set = len(atom_idx_list)
dataset_array = np.concatenate([preprocess_atom_set(rna, atom_idx_list[i]) for i in tqdm(range(n_atom_idx_set))])
np.random.seed(1)
np.random.shuffle(dataset_array)
n_set = len(dataset_array)
split_idx = math.ceil(n_set/2)
train_dataset = tf.data.Dataset.from_tensor_slices(dataset_array[:split_idx])
test_dataset = tf.data.Dataset.from_tensor_slices(dataset_array[split_idx:])
train_dataset.save("VAE/dataset/ssRNA_train_dataset")
test_dataset.save("VAE/dataset/ssRNA_test_dataset")

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [26]:
# DEBUGGING SECCTION
atom_idx = atom_idx_list[0]
pair_meshgrid = np.meshgrid(atom_idx, np.transpose(atom_idx))
pair_grid = np.dstack((pair_meshgrid[0], pair_meshgrid[1]))
pair_distance_array_frames = compute_and_pack_distance_in_array_frames(rna, pair_grid)
formatted_pair_distance_array_frames = np.array(
    [trunc_or_pad_distance_array(frame, maxlen = 100) 
     for frame in pair_distance_array_frames])

NameError: name 'compute_and_pack_distance_in_array_frames' is not defined